In [3]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Tout le travail API


Comme certaines cellules prennent longtemps, on a gardé en csv certains résultats, pour éviter d'avoir des erreurs mais tout de même garder une trace de ce qui a été fait. On a mis certaines cellules en commentaires. Même si de toute manière les clefs auront expirées 

In [4]:
#API keys for the Riot Games API
api_key = "RGAPI-6e178b98-ed70-42db-be1b-1c60e459366f" #Compte Slylio 
api_key2 = "RGAPI-1fbe27bd-678c-416f-a37c-bffeea23508d" #Compte emilio
api_key3 = "RGAPI-26cbf7f1-67e3-43ca-ac1a-5e040aa115a0" #Compte projetML
api_key5 = "RGAPI-a5972672-c708-4f9d-9fa3-2908e0d52c0e" #Comte alex1
api_key6 = "RGAPI-9dd5b684-91d0-4a69-bf6e-c945b31129f4" #Compte alex2
api_key7 = "RGAPI-46165a21-b349-4a50-9e4b-9bf71bc5d784" #Compte louis

In [5]:
"""_summary_
    This function returns the summonerId of master + ranks players in europe west.
    
    _description_
    This function returns the summonerId of the player by using the API key.
    
    _parameters_
    api_key : str
        The API key that is used to authenticate the request
"""

def get_summonerId(api_key):
    challenger_url = 'https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challanger ENDPOINT
    grandmaster_url = 'https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Grandmaster EDNPOINT
    master_url = 'https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Master ENDPOINT

    league_url_lists = [challenger_url, grandmaster_url, master_url] # Creating a list with the 3 endpoints (master, grandmaster and challenger)

    df_list = [] # A list that will store the response from the get request (league_df)

    for url in league_url_lists: # Loop interation through league_url_lists
        resp = requests.get(url) # Sending a get request to the url
        league_df = pd.json_normalize(resp.json()['entries'])
        df_list.append(league_df)
    
    final_league_df = pd.concat(df_list) # Concatenating the stored information in league_url_lists to create the final_league_df

    return final_league_df

In [6]:
league_df = get_summonerId(api_key)
print('league_df has information about {} players between master and challenger elo'.format(league_df.shape[0]))

league_df has information about 18603 players between master and challenger elo


In [7]:
league_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,HZ76qM6ie7IhyfCBZhyHKuLBomSyzd0w6bHWs5JMUM77aJ...,,1311,I,174,90,False,False,False,False
1,vCckDc7g1dhjbFxwcECg68mvYWbgRbKeiB9wFPAlF4tyYPGF,Skillshot Lander,1036,I,337,280,True,False,False,False
2,v1c4ckEpCnutqEJ29WvzHGrr9nQ8a3_r2RxxKU4HnuQjVv...,jonas2246,887,I,124,85,False,False,False,False
3,olbE76G7T-Lph3LKtqEspWawaB7b0RTwkulbe7OPd7O1RbQ,TakeSet sama,1128,I,247,194,True,False,False,False
4,blhkbwZQro8FzMXbJ4VmhvnHkVuxhhTb70AQW5hssFw7KJ...,Musa51,856,I,137,96,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
17598,-j8RelGMgRAi8DDHKI92EtavS_Gfub_tU91AjTFEokmLQvw,Markoreformedarc,253,I,188,183,True,False,False,True
17599,OkhtZi2VOEVyNb9KjR1tF0MrqKF8euQDXTEpTmEMa0YUKxc,trying stuff,165,I,33,15,False,False,False,False
17600,wDCw0G7brYzme-ZKdXFuYQIllXbbWO6zu6HuknYc_AAOq9k,Atheist Rengar,193,I,78,66,False,False,False,False
17601,mNdCOFT6TQBpIvPzVXxRslakpGjFbebrsEG6T-CJLD-kc64,BS Undaq,472,I,196,176,True,False,False,True


In [8]:
def get_summonerinfo_by_summonerId(df, api_key):
    
    summoner_list = [] # list that will store the each summoner_info DataFrame

    for i in range(len(df)): # len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        # Creating a dynamic api_ulr for each register in the summonerId column using iloc
        api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
        print(api_url)
        # Repeating the process we did at the begining
        resp = requests.get(api_url)
        print(resp.status_code)

        # if, elif and while true to deal with riot api rate limit
        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 10 second') #approximate 10 second wait before trying again
                    time.sleep(10)

                    # Trying again
                    print(api_url)
                    api_url = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break

        summoner_info = pd.json_normalize(resp.json()) # The same process we did at the beggining to create the DataFrame
        summoner_list.append(summoner_info) # Appending the DataFrame generated (summoner_info) into the list "summoner_list"
        
    final_summoner_info = pd.concat(summoner_list) # Using pd.concat to concatenate the results stored in the list "summoner_list"
            
    return final_summoner_info
    

In [9]:
"""
summoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)
summoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)
summoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)
summoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)
summoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)
summoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])
summoner_info.drop_duplicates(subset=['accountId'], inplace=True)
summoner_info.to_csv('summoner_info.csv', index=False)
"""

"\nsummoner_info = get_summonerinfo_by_summonerId(league_df[:1500], api_key)\nsummoner_info2 = get_summonerinfo_by_summonerId(league_df[1500:3000], api_key)\nsummoner_info3 = get_summonerinfo_by_summonerId(league_df[3000:5500], api_key)\nsummoner_info4 = get_summonerinfo_by_summonerId(league_df[5500:8000], api_key)\nsummoner_info5 = get_summonerinfo_by_summonerId(league_df[8000:10500], api_key)\nsummoner_info = pd.concat([summoner_info, summoner_info2, summoner_info3, summoner_info4, summoner_info5])\nsummoner_info.drop_duplicates(subset=['accountId'], inplace=True)\nsummoner_info.to_csv('summoner_info.csv', index=False)\n"

In [10]:
def get_matchids_by_puuid(df, api_key):

    matchids_list = []

    for i in range(len(df)):

        api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.json())

        if resp.status_code == 200:
            matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
            matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 100 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count=50&api_key={}'.format(df['puuid'].iloc[i], api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                    if resp.status_code == 200:
                        matchids = pd.DataFrame(resp.json(), columns=['matchid']) 
                        matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
                        print('limit cost resolve')
                        break
                else:
                    break

    matchids_df = pd.concat(matchids_list) # Using pd.concat to concatenate the results stored in the "matchids_list"
    
    return matchids_df


In [11]:
#Remove duplicates
"""
size_before = matchids_df.shape[0]
matchids_df = matchids_df.drop_duplicates()
size_after = matchids_df.shape[0]
print('Removed {} duplicates'.format(size_before - size_after))
print('matchids_df has information about {} matches'.format(matchids_df.shape[0]))
"""

"\nsize_before = matchids_df.shape[0]\nmatchids_df = matchids_df.drop_duplicates()\nsize_after = matchids_df.shape[0]\nprint('Removed {} duplicates'.format(size_before - size_after))\nprint('matchids_df has information about {} matches'.format(matchids_df.shape[0]))\n"

In [12]:
#matchids_df.to_csv('matchids.csv', index=False)

In [13]:
#matchids_df.reset_index(inplace=True)
#matchids_df.drop(columns = 'index', inplace = True )
#matchids_df

In [14]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}/timeline?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json())
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [15]:
from concurrent.futures import ThreadPoolExecutor

In [16]:
matchs_ids = pd.read_csv('matchids.csv')

In [17]:
matchs_ids

,matchid
0,EUW1_6872766682
1,EUW1_6872721234
2,EUW1_6872682077
3,EUW1_6872636755
4,EUW1_6872581293
...,...
237638,EUW1_6811998207
237639,EUW1_6811267636
237640,EUW1_6811199346
237641,EUW1_6810916271


In [18]:
def multi_get_match_info(matchs_ids, api_keys):
    with ThreadPoolExecutor() as executor:
        futures = []
        matchs_info_df = []
        for i in range(len(api_keys)):
            future = executor.submit(get_match_info_by_matchId, np.array_split(matchs_ids, len(api_keys))[i], api_keys[i])
            futures.append(future)
        for future in futures:
            matchs_info_df.append(future.result())
    return pd.concat(matchs_info_df)

    
#Create a 5Gb file
df_matchs_info1 = multi_get_match_info(matchs_ids[:100], [api_key, api_key2, api_key3, api_key5, api_key6, api_key7])

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [19]:
df_matchs_info1.shape
df_matchs_info1

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants
0,2,EUW1_6872766682,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
0,2,EUW1_6872721234,[nFVx2BkBRefMe_OoGq9ATC9zKDtMRxC9s8PGd8oTu61aL...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'nFVx2BkBRefMe_..."
0,2,EUW1_6872682077,[lreOyIqCn3xo20ipRzSwUTvwqW8eFxA0l8iXGni3OM9Rp...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'lreOyIqCn3xo20..."
0,2,EUW1_6872636755,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
0,2,EUW1_6872581293,[wCdvPlukkQSkihDiNIfb5Bkez12fl-hScsUtlhCWgJ1Ib...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'wCdvPlukkQSkih..."
...,...,...,...,...,...,...,...,...
0,2,EUW1_6853619933,[TKbrSdR6OcgB6LPDKddFexGVpgoffJ3n58Es2IOmf1Hh2...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710269234617, ...",6853619933,"[{'participantId': 1, 'puuid': 'TKbrSdR6OcgB6L..."
0,2,EUW1_6853580203,[qKPitFo0h5bnIXvRvrlYCgS_NE3vm7vxQlLWH0d5f6nYq...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710267734792, ...",6853580203,"[{'participantId': 1, 'puuid': 'qKPitFo0h5bnIX..."
0,2,EUW1_6853508233,[-D4g7VFc92-8T8PFMelWK1cuwbWFWwxaSTcMc9Umla0Bt...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710264489863, ...",6853508233,"[{'participantId': 1, 'puuid': '-D4g7VFc92-8T8..."
0,2,EUW1_6853320591,[T3Coe7tP4RwxgUX7WXD5ucFRnG0kMeEpw42sVGkBfK85F...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710253783590, ...",6853320591,"[{'participantId': 1, 'puuid': 'T3Coe7tP4RwxgU..."


In [20]:
matchs_info_df_index = df_matchs_info1.reset_index()
matchs_info_df_index

,index,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.frameInterval,info.frames,info.gameId,info.participants
0,0,2,EUW1_6872766682,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711467506447, ...",6872766682,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
1,0,2,EUW1_6872721234,[nFVx2BkBRefMe_OoGq9ATC9zKDtMRxC9s8PGd8oTu61aL...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711465572396, ...",6872721234,"[{'participantId': 1, 'puuid': 'nFVx2BkBRefMe_..."
2,0,2,EUW1_6872682077,[lreOyIqCn3xo20ipRzSwUTvwqW8eFxA0l8iXGni3OM9Rp...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711463416185, ...",6872682077,"[{'participantId': 1, 'puuid': 'lreOyIqCn3xo20..."
3,0,2,EUW1_6872636755,[avW8BZTi9HCQIiloW2teMJKzyx2yDpkW1qwE3K4lf8k3N...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711460683572, ...",6872636755,"[{'participantId': 1, 'puuid': 'avW8BZTi9HCQIi..."
4,0,2,EUW1_6872581293,[wCdvPlukkQSkihDiNIfb5Bkez12fl-hScsUtlhCWgJ1Ib...,GameComplete,60000,"[{'events': [{'realTimestamp': 1711457894967, ...",6872581293,"[{'participantId': 1, 'puuid': 'wCdvPlukkQSkih..."
...,...,...,...,...,...,...,...,...,...
95,0,2,EUW1_6853619933,[TKbrSdR6OcgB6LPDKddFexGVpgoffJ3n58Es2IOmf1Hh2...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710269234617, ...",6853619933,"[{'participantId': 1, 'puuid': 'TKbrSdR6OcgB6L..."
96,0,2,EUW1_6853580203,[qKPitFo0h5bnIXvRvrlYCgS_NE3vm7vxQlLWH0d5f6nYq...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710267734792, ...",6853580203,"[{'participantId': 1, 'puuid': 'qKPitFo0h5bnIX..."
97,0,2,EUW1_6853508233,[-D4g7VFc92-8T8PFMelWK1cuwbWFWwxaSTcMc9Umla0Bt...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710264489863, ...",6853508233,"[{'participantId': 1, 'puuid': '-D4g7VFc92-8T8..."
98,0,2,EUW1_6853320591,[T3Coe7tP4RwxgUX7WXD5ucFRnG0kMeEpw42sVGkBfK85F...,GameComplete,60000,"[{'events': [{'realTimestamp': 1710253783590, ...",6853320591,"[{'participantId': 1, 'puuid': 'T3Coe7tP4RwxgU..."


In [21]:
dataset = pd.DataFrame()
for i in range(len(matchs_info_df_index)):
    if (type(matchs_info_df_index["info.frames"][i])==list) : 
        last_event = matchs_info_df_index["info.frames"][i][-1]["events"][-1] #Get the last event of the match
        winning_team = int((last_event["winningTeam"] - 100) / 100) #Get the winning team of the match and convert it from 100 or 200 to 0 or 1
        end = pd.DataFrame({"winning_team": [winning_team]})
        
        match_data_stats = pd.json_normalize(pd.DataFrame(matchs_info_df_index["info.frames"][i])["participantFrames"]) #Get the participantFrames data
        match_data_stats.columns = match_data_stats.columns.str.replace(r'^\d+\.', '', regex=True) #Remove the number at the beginning of the columns names
        
        #Get the stats of the blue and red teams
        team1_stats = match_data_stats.iloc[:, 1:240] 
        team2_stats = match_data_stats.iloc[:, 240:] 
        
        #Sum the stats of the blue and red teams
        team1_stats = team1_stats.groupby(team1_stats.columns, axis=1).sum().add_prefix("Team1.")
        team2_stats = team2_stats.groupby(team2_stats.columns, axis=1).sum().add_prefix("Team2.")
        
        match_data_stats_teams = pd.concat([team1_stats, team2_stats], axis=1) #Concatenate the two teams stats to get
        
        if len(match_data_stats_teams) > 15: #If the match has more than 15 minutes, else we skip it
            match_data_stats_teams_15 = match_data_stats_teams.loc[15] #Get the stats of the match at 15 minutes (MODIFY THIS TO GET THE STATS AT 10 MINUTES OR 20 MINUTES OR 25 MINUTES ETC.)
            match_data_stats_teams_15= pd.DataFrame(match_data_stats_teams_15).transpose().reset_index().drop(["index"], axis=1) #Transform the data to a DataFrame
            pd.concat([end, match_data_stats_teams_15], axis=1) 
            dataset = pd.concat([dataset, pd.concat([end, match_data_stats_teams_15], axis=1)], axis = 0)
dataset

,winning_team,Team1.championStats.abilityHaste,Team1.championStats.abilityPower,Team1.championStats.armor,Team1.championStats.armorPen,Team1.championStats.armorPenPercent,Team1.championStats.attackDamage,Team1.championStats.attackSpeed,Team1.championStats.bonusArmorPenPercent,Team1.championStats.bonusMagicPenPercent,...,Team2.goldPerSecond,Team2.jungleMinionsKilled,Team2.level,Team2.minionsKilled,Team2.participantId,Team2.position.x,Team2.position.y,Team2.timeEnemySpentControlled,Team2.totalGold,Team2.xp
0,1,0,177,352,0,0,628,817,0,0,...,50,100,47,421,40,58227,25880,1049805,24603,29860
0,1,0,185,446,0,0,551,618,0,0,...,50,95,46,402,40,44645,41057,395210,27729,30281
0,1,0,186,466,0,0,603,826,0,0,...,50,104,49,389,40,39313,27933,730043,31166,31966
0,0,0,195,307,0,0,746,945,0,0,...,50,107,48,385,40,43743,50106,957390,30368,32863
0,1,0,230,348,0,30,759,635,0,0,...,50,83,46,370,40,49966,24051,725235,28765,29713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,0,320,295,0,0,631,709,0,0,...,50,66,45,388,40,41522,40695,569400,27947,27745
0,0,0,210,328,0,0,679,794,0,0,...,50,84,45,382,40,41179,33050,568897,27176,27952
0,1,0,244,290,0,0,609,689,0,0,...,50,103,49,401,40,26874,39522,844558,32129,33148
0,1,0,173,311,0,0,586,710,0,0,...,50,99,48,393,40,43194,42981,775867,27824,31246


In [23]:
dataset.to_csv("data_15minX.csv", index = False) #Export data into csv to use it in the preprocessing file.